In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

/Users/sajanakadangara/code_base/ljmu_research/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
df = pd.read_excel("data/final_data.xlsx")

In [6]:
df.head()

,source,source_country,source_latitude,source_longitude,destination,shipping_mode,dest_lat,dest_lon,destination_country
0,Caguas,Puerto Rico,18.359064,-66.370575,Viena,Standard Class,48.2082,16.3738,Austria
1,Caguas,Puerto Rico,18.359064,-66.370575,Viena,Standard Class,48.2082,16.3738,Austria
2,Caguas,Puerto Rico,18.359064,-66.370575,Viena,Standard Class,48.2082,16.3738,Austria
3,Caguas,Puerto Rico,18.359064,-66.370575,Viena,Standard Class,48.2082,16.3738,Austria
4,Caguas,Puerto Rico,18.359064,-66.370575,Viena,Standard Class,48.2082,16.3738,Austria


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482777 entries, 0 to 482776
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   source_city       482777 non-null  object 
 1   source_country    482777 non-null  object 
 2   source_latitude   482777 non-null  float64
 3   source_longitude  482777 non-null  float64
 4   dest_city         482777 non-null  object 
 5   shipping_mode     482777 non-null  object 
 6   dest_lat          482777 non-null  float64
 7   dest_lon          482777 non-null  float64
 8   dest_country      482777 non-null  object 
dtypes: float64(4), object(5)
memory usage: 33.1+ MB


In [ ]:
df['source_country'] = df['source_country'].apply(lambda x: x.lower())
df['destination_country'] = df['destination_country'].apply(lambda x: x.lower())

df['source'] = df['source'].apply(lambda x: x.lower())
df['destination'] = df['destination'].apply(lambda x: x.lower())

In [ ]:
# Helper function to compute distance and assign modes/vehicles
def assign_transport(row):
    origin = (row['source_latitude'], row['source_longitude'])
    dest = (row['dest_lat'], row['dest_lon'])
    distance_km = geodesic(origin, dest).km
    shipping_mode = row['shipping_mode']
    same_country = row['source_country'] == row['destination_country']

    # Logic to assign transport category and transportation mode
    # Emission rate = grams CO2 per km
    # Electric Van estimated at 40 g/km (average grid electricity emissions)
    if not same_country:
        if shipping_mode in ["Same Day", "First Class"]:
            transport_category = "Air"
            transport_mode = "Air Freight"
            emission_rate = 600
        else:
            transport_category = "Sea"
            transport_mode = "Container Ship"
            emission_rate = 10
    elif shipping_mode == "Same Day" and distance_km > 1000:
        transport_category = "Air"
        transport_mode = "Air Freight"
        emission_rate = 600
    elif distance_km > 1000:
        transport_category = "Rail"
        transport_mode = "Cargo Train"
        emission_rate = 50
    elif distance_km > 500:
        transport_category = "Road"
        transport_mode = "Heavy Truck"
        emission_rate = 500
    elif distance_km > 100:
        transport_category = "Road"
        transport_mode = "Medium Truck"
        emission_rate = 300
    elif shipping_mode in ["Same Day", "First Class"]:
        transport_category = "Road"
        transport_mode = "Small Van"
        emission_rate = 180
    else:
        transport_category = "Road"
        transport_mode = "Electric Van"
        emission_rate = 40  # # realistic grid-powered emissions

    emission_kg = distance_km * emission_rate / 1000

    return pd.Series([distance_km, transport_category, transport_mode, emission_kg])

In [ ]:
# calculate 4 new variables using the above helper function
# distance, transport mode, vehicle type and emission
df[['distance', 'transport_category', 'transportation_mode', 'emission']] = df.apply(assign_transport, axis=1)

In [ ]:
# export data with all variables for detailed EDA
df.to_excel("data/input_data.xlsx",index=False)

In [ ]:
# select the final features use for VAE model, and drop others

'''final features selected:
1. source country
2. destination country
3. source city
4. dest city
5. distance
6. transportation_mode
7. emission
'''
df.drop(columns=["source_latitude","source_longitude","dest_lat","dest_lon","transport_category","shipping_mode"
 ], inplace=True)

In [ ]:
# drop duplicate values
df_no_duplicates = df.drop_duplicates()

In [ ]:
df_no_duplicates.to_excel("data/supply_chain_data.xlsx",index=False)